<a href="https://colab.research.google.com/github/JOYAL886/alzheimer-predictor/blob/main/Alzheimers_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
folder = "/content/drive/My Drive/Colab Notebooks/Data/"

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import autocast, GradScaler
import timm
from google.colab import drive


In [5]:
# Path to dataset
dataset_path = "/content/drive/MyDrive/Colab Notebooks/Data/"

In [6]:
# Define image transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # MRI is likely grayscale
    transforms.Resize((224, 224)),                # Required by most models
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Load dataset
full_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Remove missing/broken image paths
import os
full_dataset.samples = [(p, l) for p, l in full_dataset.samples if os.path.exists(p)]

# Print class names
print("Classes:", full_dataset.classes)


Classes: ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']


In [7]:
# Split into 70% train, 15% val, 15% test
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_and_evaluate(model_name='efficientnet_b0', num_epochs=2):
    print(f"\nTraining Model: {model_name}\n" + "="*50)

    model = timm.create_model(model_name, pretrained=True, num_classes=4).to(device)
    if hasattr(torch, 'compile'):
        model = torch.compile(model)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)
    scaler = GradScaler()

    train_acc_list, val_acc_list = [], []

    def calc_accuracy(loader):
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                with autocast():
                    preds = model(x)
                _, predicted = torch.max(preds, 1)
                correct += (predicted == y).sum().item()
                total += y.size(0)
        return 100 * correct / total

    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast():
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

        train_acc = 100 * correct / total
        val_acc = calc_accuracy(val_loader)
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        scheduler.step()

        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss/len(train_loader):.4f}, "
              f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

    torch.save(model.state_dict(), f"{model_name}_alzheimers.pth")
    return model


In [9]:
pip install tqdm


In [10]:
from tqdm import tqdm  # add this at the top

def train_and_evaluate(model_name='efficientnet_b0', num_epochs=1):
    print(f"\nTraining Model: {model_name}\n" + "="*50)

    model = timm.create_model(model_name, pretrained=True, num_classes=4).to(device)

    if hasattr(torch, 'compile') and torch.cuda.is_available():
        model = torch.compile(model)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

    # Handle mixed precision safely
    if torch.cuda.is_available():
      scaler = torch.cuda.amp.GradScaler()
      autocast = lambda: torch.amp.autocast(device_type='cuda')
    else:
      scaler = torch.amp.GradScaler(enabled=False)
      autocast = lambda: torch.amp.autocast(device_type='cpu')


    train_acc_list, val_acc_list = [], []

    def calc_accuracy(loader):
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(device), y.to(device)
                with autocast():
                    preds = model(x)
                _, predicted = torch.max(preds, 1)
                correct += (predicted == y).sum().item()
                total += y.size(0)
        return 100 * correct / total

    for epoch in range(num_epochs):
        print(f"\n🔄 Starting Epoch {epoch+1}/{num_epochs}...\n" + "-"*40)

        model.train()
        total_loss, correct, total = 0.0, 0, 0

        # ✅ Add tqdm progress bar here
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for x, y in progress_bar:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            with autocast():
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            _, pred = torch.max(out, 1)
            correct += (pred == y).sum().item()
            total += y.size(0)

            # ✅ Update progress bar with loss
            progress_bar.set_postfix(loss=loss.item())

        train_acc = 100 * correct / total
        val_acc = calc_accuracy(val_loader)
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        scheduler.step()

        print(f"✅ Epoch {epoch+1} Complete - Loss: {total_loss/len(train_loader):.4f}, "
              f"Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

    torch.save(model.state_dict(), f"{model_name}_alzheimers.pth")
    print(f"\n📦 Model saved as '{model_name}_alzheimers.pth'")
    return model


In [11]:
print("Using device:", device)


Using device: cuda


In [12]:
model = train_and_evaluate()



Training Model: efficientnet_b0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

/tmp/ipython-input-1177633564.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



🔄 Starting Epoch 1/1...
----------------------------------------


Epoch 1/1:   0%|          | 0/521 [00:00<?, ?it/s]W0808 19:20:40.496000 189 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


✅ Epoch 1 Complete - Loss: 0.3370, Train Acc: 89.39%, Val Acc: 99.52%

📦 Model saved as 'efficientnet_b0_alzheimers.pth'


In [13]:
torch.save(model.state_dict(), "alzheimers_model.pth")


In [14]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            with autocast():
                out = model(x)
            _, preds = torch.max(out, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

test_acc = evaluate(model, test_loader)
print(f"\n✅ Final Test Accuracy: {test_acc:.2f}%")


/tmp/ipython-input-2117385335.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipython-input-2117385335.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():



✅ Final Test Accuracy: 99.41%
